### Configuración de los punstos de acceso de los bucket para aplicaciones

#### Problema
Tienes un bucket de S3 y dos aplicaciones. Necesita conceder acceso de lectura/escritura a una de sus aplicaciones y acceso de sólo lectura a otra aplicación. No desea utilizar las políticas de los cubos de S3, ya que espera tener que añadir aplicaciones adicionales con requisitos de seguridad más precisos en el futuro.

#### Solución
Cree dos puntos de acceso de S3 y aplique una política que conceda las acciones *S3:PutObject* y *S3:GetObject* a uno de los puntos de acceso y *S3:GetObject* al otro punto de acceso. A continuación, configure su aplicación para utilizar el nombre DNS del punto de acceso respectivo.
<br>
<br>
<img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781492092599/files/assets/awsc_0309.png" width="500">


In [1]:
import boto3
import json
import random


region_aws = 'us-east-1'

s3 = boto3.resource('s3')
s3_client = boto3.client('s3')
s3control = boto3.client('s3control', region_name=region_aws)
ec2 = boto3.resource('ec2', region_name=region_aws)
ec2_client = boto3.client('ec2', region_name=region_aws)

In [2]:
# Create a private VPC
vpc = ec2.create_vpc(CidrBlock='10.10.0.0/16')

In [3]:
# Crear 2 subredes para la VPC
vpc_subnet_private1 = ec2.create_subnet(
    CidrBlock='10.10.1.0/24',
    VpcId=vpc.id,
    AvailabilityZone=region_aws+'a'
)
vpc_subnet_private2 = ec2.create_subnet(
    CidrBlock='10.10.2.0/24', 
    VpcId=vpc.id,
    AvailabilityZone=region_aws+'b'
)

In [4]:
# Create tags
vpc_subnet_private1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-Private-1"}])
vpc_subnet_private2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-Private-2"}])

[ec2.Tag(resource_id='subnet-033e41fe395a7d74e', key='Name', value='AWSCookBookVPC-Private-2')]

In [5]:
# Crear routables para la VPC
vpc_route_table_private_1 = ec2.create_route_table(VpcId=vpc.id)
vpc_route_table_private_1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-Private-Route-Table-a"}])
vpc_route_table_private_2 = ec2.create_route_table(VpcId=vpc.id)
vpc_route_table_private_2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-Private-Route-Table-b"}])

[ec2.Tag(resource_id='rtb-09cd083101da76de0', key='Name', value='AWSCookBookVPC-Private-Route-Table-b')]

In [6]:
# Asoociar la tabla de rutas a la subred privada de VPC1
vpc_route_table_private_1.associate_with_subnet(SubnetId=vpc_subnet_private1.id)
vpc_route_table_private_2.associate_with_subnet(SubnetId=vpc_subnet_private2.id)

ec2.RouteTableAssociation(id='rtbassoc-088e2ee51e3d0ab9e')

In [8]:
# Habilitar la resolución de DNS en la VPC
vpc.modify_attribute(EnableDnsSupport={'Value': True})
vpc.modify_attribute(EnableDnsHostnames={'Value': True})

{'ResponseMetadata': {'RequestId': '4f0eb560-7eb3-4789-bf13-72bc42e6acfc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4f0eb560-7eb3-4789-bf13-72bc42e6acfc',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '237',
   'date': 'Mon, 17 Oct 2022 14:42:55 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [9]:
print("En la terminal ejecute el siguiente comando:")
print("    python create_ec2_ssm.py --vpc {} --subnet {} --tag Cookbook-SSM-Instance-1".format(vpc.id, vpc_subnet_private1.id))

En la terminal ejecute el siguiente comando:
    python create_ec2_ssm.py --vpc vpc-0b26bc7d95f5b8547 --subnet subnet-0139f19e0ecd931ad --tag Cookbook-SSM-Instance-1


In [10]:
print("También ejecute el siguiente comando:")
print("    python create_ec2_ssm.py --vpc {} --subnet {} --tag Cookbook-SSM-Instance-2".format(vpc.id, vpc_subnet_private2.id))

También ejecute el siguiente comando:
    python create_ec2_ssm.py --vpc vpc-0b26bc7d95f5b8547 --subnet subnet-033e41fe395a7d74e --tag Cookbook-SSM-Instance-2


In [11]:
# Create a bucket
bucket_name = 'cookbook-{}'.format(random.randint(10000, 1000000))
bucket = s3.create_bucket(Bucket=bucket_name)
print("Bucket name: {}".format(bucket_name))

# Por defecto cuando creamos un bucket, las opciones de **Block all public access** (Bloquear todo el acceso público) 
# están desactivadas por defecto.

response = s3_client.put_public_access_block(
        Bucket=bucket_name,
        PublicAccessBlockConfiguration={
            'BlockPublicAcls': True,
            'IgnorePublicAcls': True,
            'BlockPublicPolicy': True,
            'RestrictPublicBuckets': True
        }
)

Bucket name: cookbook-558513


In [25]:
# Get account ID
account_id = boto3.client('sts').get_caller_identity().get('Account')

In [26]:
app1_ap_name = 'cookbook305-app-1'
app2_ap_name = 'cookbook305-app-2'

In [27]:
# Create an access point for Application 1
response_1 = s3control.create_access_point(
    AccountId=account_id,
    Name=app1_ap_name,
    Bucket=bucket_name,
    PublicAccessBlockConfiguration={
        'BlockPublicAcls': False,
        'IgnorePublicAcls': False,
        'BlockPublicPolicy': False,
        'RestrictPublicBuckets': False
    },
    VpcConfiguration={
        'VpcId': vpc.id,
    }
)

# Create an access point for Application 2
response_2 = s3control.create_access_point(
    AccountId=account_id,
    Name=app2_ap_name,
    Bucket=bucket_name,
    PublicAccessBlockConfiguration={
        'BlockPublicAcls': False,
        'IgnorePublicAcls': False,
        'BlockPublicPolicy': False,
        'RestrictPublicBuckets': False
    },
    VpcConfiguration={
        'VpcId': vpc.id,
    }
)

In [28]:
# Get EC2 instance profile
ec2_instance_profile = ec2_client.describe_iam_instance_profile_associations()['IamInstanceProfileAssociations'][0]['IamInstanceProfile']['Arn']
ec2_instance_profile_name = ec2_instance_profile.split("/")[-1]

In [48]:
app1_policy = {
  "Version":"2012-10-17",
  "Statement": [
  {
    "Effect": "Allow",
    "Principal": "*",
    "Action": [
      "s3:GetObject",
      "s3:PutObject"
      ],
    "Resource": "arn:aws:s3:{}:{}:accesspoint/{}/object/*".format(region_aws, account_id, app1_ap_name)
  }]
}

In [54]:
# Put the policy you created on the access point for Application 1:
response = s3control.put_access_point_policy(
    AccountId=account_id,
    Name=app1_ap_name,
    Policy=json.dumps(app1_policy)
)

In [38]:
app2_policy = {
  "Version":"2012-10-17",
  "Statement": [
  {
    "Effect": "Allow",
    "Principal": "*",
    "Action": [
      "s3:GetObject"
      ],
    "Resource": "arn:aws:s3:{}:{}:accesspoint/{}/object/*".format(region_aws, account_id, app2_ap_name)
  }]
}

In [39]:
# Put the policy you created on the access point for Application 1:
response = s3control.put_access_point_policy(
    AccountId=account_id,
    Name=app2_ap_name,
    Policy=json.dumps(app2_policy)
)

In [24]:
# Upload a file to the bucket
s3.Object(bucket_name, 'cookbook305-app-1/object/Recipe305Test.txt').put(Body=open('sample-file.txt', 'rb'))

{'ResponseMetadata': {'RequestId': '1W5V7X8EM39ANZK8',
  'HostId': 'ODWb08lJg5FSlqN5YBpwP+eXrPikKptiq+WUR13e7M/Rubyb5N9aHQnDS02OBdpHXydfBytzl4E=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'ODWb08lJg5FSlqN5YBpwP+eXrPikKptiq+WUR13e7M/Rubyb5N9aHQnDS02OBdpHXydfBytzl4E=',
   'x-amz-request-id': '1W5V7X8EM39ANZK8',
   'date': 'Mon, 17 Oct 2022 15:01:48 GMT',
   'etag': '"5c2c4b2214e4768ff9b6ee021664bda8"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"5c2c4b2214e4768ff9b6ee021664bda8"'}